# Preprocess
### Dataset 전부 불러오기

In [1]:
import datasets

total = datasets.load_from_disk("./data/concat_data/total_dataset")
train_data = datasets.load_from_disk("./data/arrow_datasets/00.train_essays.csv")
data_1 = datasets.load_from_disk("./data/arrow_datasets/01.ai_generated.csv")
data_2 = datasets.load_from_disk("./data/arrow_datasets/02.train_essays_7_prompts.csv")
data_3 = datasets.load_from_disk("./data/arrow_datasets/03.persuade15_claude_instant1.csv")
data_4 = datasets.load_from_disk("./data/arrow_datasets/04.train_drcat_04.csv")
data_5 = datasets.load_from_disk("./data/arrow_datasets/05.train_external_drcat_02.csv")
data_6 = datasets.load_from_disk("./data/arrow_datasets/06.argugpt.csv")
data_7 = datasets.load_from_disk("./data/arrow_datasets/07.persuade_2.0_human_scores_demo_id_github.csv")
data_8 = datasets.load_from_disk("./data/arrow_datasets/09.llama70b_and_falcon180b.csv")
data_9 = datasets.load_from_disk("./data/arrow_datasets/10.sakibsh_dataset.csv")
data_10 = datasets.load_from_disk("./data/arrow_datasets/11.ielts_writing_dataset.csv")

/data/bart/temp_workspace/AI-gen-detector/.venv/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


### train_essays는 모든 데이터를 다 넣을 것이다. 따라서, 외의 데이터들만 합쳐준다

In [2]:
external_dataset = datasets.concatenate_datasets([data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10])

### 필요시에 따라 저장

In [ ]:
external_dataset.save_to_disk("./data/concat_data/only_external")

## 여기를 넘어가기 전에, text_dedup을 이용하여, ex_minhash_dedup을 생성해줘야한다.

### 0과 1의 비율이 동일하면서, train_essays는 전부 포함될 수 있도록 작업

In [3]:
ex_data = datasets.load_from_disk("./data/concat_data/ex_minhash_dedup_99")
ex_human = ex_data.filter(lambda x:x["generated"] == 0)
ex_llm = ex_data.filter(lambda x:x["generated"] == 1)

train_llm = train_data.filter(lambda x:x["generated"]==1)
train_human = train_data.filter(lambda x:x["generated"]==0)

### 두 개수 중에 적은 쪽을 택한다. (Under Sampling)

In [4]:
min_val = min(len(ex_llm)+len(train_llm), len(ex_human)+len(train_human))
print(min_val)

19323


### train_data만큼은 다 들어가야하니, 제외하고 샘플링한다.

In [5]:
human_split = ex_human.shuffle(seed=42).select(range(min_val-len(train_human)))
llm_split = ex_llm.shuffle(seed=42).select(range(min_val-len(train_llm)))

### train_data를 마지막에 합쳐주면, 비율은 동일하면서, 개수는 딱 맞게 된다.

In [6]:
total_human = datasets.concatenate_datasets([train_human, human_split])
total_llm = datasets.concatenate_datasets([train_llm, llm_split])

assert len(total_llm) == len(total_human) == min_val, "데이터 숫자처리 뭔가 오류남"

/data/bart/temp_workspace/AI-gen-detector/.venv/lib/python3.10/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]


### 최종 데이터 만드는 작업

In [7]:
total_data = datasets.concatenate_datasets([total_human, total_llm])

### 필요시에 저장

In [54]:
total_data.save_to_disk("./data/concat_data/under_sample_99")

Saving the dataset (0/1 shards):   0%|          | 0/38646 [00:00<?, ? examples/s]

# K-fold data를 만드는 과정

In [8]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True)


#### 홀짝 여부에 따라 분포는 최대 1개까지 달라질 수 있다.

In [9]:
for fold_num, train_test_idx in enumerate(kfold.split(X=total_data, y=total_data["generated"])):
    train_idx, test_idx = train_test_idx
    train_dataset = total_data.select(train_idx)
    eval_dataset = total_data.select(test_idx)
    train_dataset.save_to_disk(f"./data/concat_data/kfold_under_sample_99/train/{fold_num}")
    eval_dataset.save_to_disk(f"./data/concat_data/kfold_under_sample_99/eval/{fold_num}")

Saving the dataset (0/1 shards):   0%|          | 0/30916 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7730 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/30917 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7729 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/30917 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7729 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/30917 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7729 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/30917 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7729 [00:00<?, ? examples/s]